In [12]:
from pyspark.mllib.feature import Word2Vec

rootpath = "/home/zachncst/classes/csc591/projects/CapstoneProject"

paths = "{0}/data/110,{0}/data/111,{0}/data/112,{0}/data/113".format(rootpath)

stopwords = "{0}/data/stopwords.txt".format(rootpath)

rdd = sc.textFile(paths)
stopwords = sc.textFile(stopwords).collect()
stopwords = set(stopwords)

stopwords

{u"'ll",
 u"'ve",
 u'able',
 u'about',
 u'above',
 u'abst',
 u'accordance',
 u'according',
 u'accordingly',
 u'across',
 u'act',
 u'actually',
 u'added',
 u'adj',
 u'affected',
 u'affecting',
 u'affects',
 u'after',
 u'afterwards',
 u'again',
 u'against',
 u'ah',
 u'all',
 u'almost',
 u'alone',
 u'along',
 u'already',
 u'also',
 u'although',
 u'always',
 u'am',
 u'among',
 u'amongst',
 u'an',
 u'and',
 u'announce',
 u'another',
 u'any',
 u'anybody',
 u'anyhow',
 u'anymore',
 u'anyone',
 u'anything',
 u'anyway',
 u'anyways',
 u'anywhere',
 u'apparently',
 u'approximately',
 u'are',
 u'aren',
 u'arent',
 u'arise',
 u'around',
 u'as',
 u'aside',
 u'ask',
 u'asking',
 u'at',
 u'auth',
 u'available',
 u'away',
 u'awfully',
 u'b',
 u'back',
 u'be',
 u'became',
 u'because',
 u'become',
 u'becomes',
 u'becoming',
 u'been',
 u'before',
 u'beforehand',
 u'begin',
 u'beginning',
 u'beginnings',
 u'begins',
 u'behind',
 u'being',
 u'believe',
 u'below',
 u'beside',
 u'besides',
 u'between',
 u'bey

In [2]:
import json
json_rdd = rdd.map(lambda x: json.loads(x)).filter(lambda x: len(x['bill']['summary']) != 0)

json_rdd.take(1)

[{u'bill': {u'current-status': [{u'details': {u'actionDate': u'2007-07-12',
      u'description': u'Passed House',
      u'displayText': u'Passed/agreed to in House: On agreeing to the resolution Agreed to by the Yeas and Nays: 221 - 196 (Roll no. 621).(text: CR H7663)',
      u'externalActionCode': u'8000'},
     u'status': u'Passed House'}],
   u'name': [u'H.Res.533'],
   u'summary': [u'Congressional Bills 110th CongressFrom the US Government Printing OfficeH Res 533 Engrossed in House EH In the House of Representatives U S July 12 2007 Resolved That upon the adoption of this resolution it shall be in order to consider in the House the bill HR 2956 to require the Secretary of Defense to commence the reduction of the number of United States Armed Forces in Iraq to a limited presence by April 1 2008 and for other purposes All points of order against the bill and against its consideration are waived except those arising under clause 9 or 10 of rule XXI The bill shall be considered as re

In [3]:
def has_status(x): 
    return len(x['bill']['current-status']) > 0

def get_status(x):
    return x['bill']['current-status'][0]['status']

statuses = json_rdd.filter(has_status)\
    .map(get_status)\
    .distinct()

#print(statuses.collect())

[u'Passed House', u'Failed Senate', u'Failed House', u'Resolving Differences', u'Passed Senate', u'Failed to pass over veto', u'Introduced']


In [62]:
def filter_status(x, status):
    return get_status(x) == status

passed_house_rdd = json_rdd.filter(has_status)\
    .filter(lambda x: filter_status(x, "Passed House") or filter_status(x, 'Passed Senate'))
    
failed_congress_rdd = json_rdd.filter(has_status)\
    .filter(lambda x: filter_status(x, "Failed House") or filter_status(x, 'Failed Senate'))

#passed_house_rdd.take(1)
#became_law_rdd.map(lambda x: x['bill']['summary']).take(1)

In [63]:
import re

def has_nums(st):
    m = re.search('\d+', st)
    return True if m else False

def list_func(ls):
    new_list = []
    
    for el in ls:
        tm = el.lower()
        if len(tm) > 3 and tm not in stopwords and not tm.isdigit() and not has_nums(tm):
            new_list.append(tm)
            
    return new_list

passed_house_rdd_strings = passed_house_rdd.map(lambda x: x['bill']['summary'][0])\
    .map(lambda row: row.split(" "))\
    .map(list_func)
    
failed_congress_rdd_strings = failed_congress_rdd.map(lambda x: x['bill']['summary'][0])\
    .map(lambda row: row.split(" "))\
    .map(list_func)


In [64]:
passed_house_rdd_individual_strings = passed_house_rdd_strings.flatMap(lambda x: x)
failed_rdd_individual_strings = failed_congress_rdd_strings.flatMap(lambda x: x)

passed_counts = passed_house_rdd_individual_strings.groupBy(lambda x: x).mapValues(lambda x : len(x)).collect()
failed_counts = failed_rdd_individual_strings.groupBy(lambda x: x).mapValues(lambda x : len(x)).collect()

In [67]:
sorted(failed_counts, key=lambda c: c[1], reverse=True)

[(u'secretary', 3982),
 (u'land', 2231),
 (u'subsection', 1684),
 (u'paragraph', 1641),
 (u'federal', 1497),
 (u'national', 1466),
 (u'program', 1414),
 (u'inserting', 1258),
 (u'striking', 1205),
 (u'management', 1203),
 (u'state', 1199),
 (u'area', 1148),
 (u'water', 1110),
 (u'amended', 1040),
 (u'food', 904),
 (u'conservation', 893),
 (u'wilderness', 879),
 (u'term', 847),
 (u'subtitle', 838),
 (u'plan', 826),
 (u'including', 804),
 (u'public', 770),
 (u'subparagraph', 762),
 (u'states', 741),
 (u'agricultural', 722),
 (u'project', 701),
 (u'fiscal', 680),
 (u'years', 676),
 (u'agriculture', 656),
 (u'year', 650),
 (u'united', 647),
 (u'producer', 634),
 (u'heritage', 620),
 (u'assistance', 612),
 (u'river', 605),
 (u'funds', 577),
 (u'title', 576),
 (u'local', 566),
 (u'eligible', 557),
 (u'entity', 548),
 (u'described', 532),
 (u'crop', 512),
 (u'protection', 503),
 (u'purposes', 503),
 (u'farm', 499),
 (u'authorized', 495),
 (u'carry', 492),
 (u'resources', 491),
 (u'determined'

In [50]:

#passed_hosue_rdd_strings = passed_house_rdd_strings\
#    .filter(lambda x: len(x) > 3)\
#    .filter(lambda x: x not in stopwords)

passed_house_model = Word2Vec().setVectorSize(10).setSeed(42).fit(passed_house_rdd_strings)

In [53]:
passed_house_model.getVectors()

{u'quotient': [0.007473447, -0.017818008, -0.04213654, 0.01859863, -0.09255969, 0.10373737, 0.12169847, 0.005900766, -0.03534558, -0.05193532], u'incident': [0.05184268, -0.2856267, 0.12963513, -0.108387224, 0.22119483, -0.21876131, 0.24899913, 0.5472177, -0.2033505, 0.030705625], u'serious': [0.15456611, -0.6472207, -0.36612108, -0.34180915, -0.011579655, -0.650435, -0.022125877, 0.28700203, 0.087921746, 0.44575387], u'comply': [0.8887132, -0.020294761, -0.023786709, 0.23853362, 0.287441, -0.3126253, 0.33381528, -0.12867087, 0.10593766, -0.060803585], u'ltdeletedgtconstruction': [-0.045553688, -0.086331636, 0.17986268, 0.089493625, -0.0076379934, 0.029898558, 0.08371115, -0.014339066, -0.013031126, -0.039126057], u'precious': [-0.060799755, -0.3164523, -0.24332005, -0.23668276, -0.13583077, -0.26785937, -0.16603062, 0.089273006, -0.059953317, 0.32654905], u'sectors': [-0.046675913, -0.25310716, -0.005920457, -0.018479144, 0.4084859, -0.1244348, 0.05592258, 0.40908536, -0.56469166, 0.1

In [68]:
synonyms = passed_house_model.findSynonyms('health', 40)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))

marketing: 0.895839500546
competitions: 0.849055429027
economically: 0.848936713079
headstones: 0.84857053943
owned: 0.835485145274
assistancesec: 0.829737000511
correspondence: 0.829716005454
benchmarking: 0.821671952711
outreach: 0.820389391379
socially: 0.819080883186
portfolio: 0.817140933161
verifications: 0.816768753436
owners: 0.815731346835
restricted: 0.811865980735
servicedisabled: 0.810779089428
disadvantaged: 0.809932793195
impacted: 0.807585636097
services: 0.806471970693
hubzone: 0.803878761882
suburban: 0.80026036257
developers: 0.794273211972
concerns: 0.790610543149
programthe: 0.781518210827
unoccupied: 0.779826924109
leveraging: 0.774815849366
connected: 0.772789161485
venture: 0.772259290809
entrepreneurial: 0.770932632346
guaranty: 0.769501666142
installing: 0.769464175597
informants: 0.769400079855
juvenile: 0.768805855615
businessessec: 0.768219629486
healthcare: 0.76436865991
subprograms: 0.763242123779
firefighter: 0.759875143384
manufacturers: 0.756787382991
e